In [ ]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import matplotlib.pyplot as plt
import seaborn as sns


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/hotels/hotel.csv")
df.drop("Unnamed: 0", inplace=True, axis=1)
df.head()

In [ ]:
df.describe().T

In our data, there are 2750 reviews but there are only 320 differen hotels. So, I will find the average rating per hotel.

In [ ]:
ratings = pd.DataFrame(df.groupby("title")['rating'].mean().sort_values(ascending=False))
ratings.head()

In [ ]:
ratings['num of ratings'] = pd.DataFrame(df.groupby("title")['rating'].count())
ratings.head()

In [ ]:
# plt.figure(figsize=(10,4))
# ratings['num of ratings'].hist()

In the table we can see that each hotel has at least 4 ratings and at most 10 ratings.

In [ ]:
# plt.figure(figsize=(10,4))
# ratings['rating'].hist()

In [ ]:
df.describe()

In [ ]:
hotel_pivot = df.pivot_table(index='user_id', columns='title', values='rating', )
hotel_pivot.head()

In [ ]:
hotel_pivot.shape

In [ ]:
ratings.sort_values("rating", ascending=False).head(15)

I have picked `"Eyna Hotel"` and `"Ankara Prenses Hotel"`. I will try to find most similar hotels.

In [ ]:
eyna_ratings = hotel_pivot['Eyna Hotel']
ankara_ratings = hotel_pivot['Ankara Prenses Hotel']
eyna_ratings

In [ ]:
ankara_ratings

In [ ]:
similar_to_eyna = hotel_pivot.corrwith(eyna_ratings, method="pearson")
similar_to_ankara = hotel_pivot.corrwith(ankara_ratings, method="pearson")

In [ ]:
similar_to_eyna.sort_values(ascending=False)

In [ ]:
similar_to_eyna.dropna(inplace=True)
similar_to_eyna.sort_values(ascending=False).head()

In [ ]:
similar_to_ankara.dropna(inplace=True)
similar_to_ankara.sort_values(ascending=False).head()

In [79]:
def hotel_recommender(name, num_hotel):
    """
    This function, in a given data set, finds the most closest hotel to the given hotel name and returns the number of hotels requested.
    name: Hotel name
    num_hotel: Number of hotels requested to be returned.
    """
    
    df = pd.read_csv("/kaggle/input/hotels/hotel.csv")
    df.drop("Unnamed: 0", inplace=True, axis=1)

    ratings = pd.DataFrame(df.groupby("title")['rating'].mean().sort_values(ascending=False))
    ratings['num of ratings'] = pd.DataFrame(df.groupby("title")['rating'].count())

    hotel_pivot = df.pivot_table(index='user_id', columns='title', values='rating', )
    
    eyna_ratings = hotel_pivot[name]


    similar_to_hotel =pd.DataFrame(hotel_pivot.corrwith(eyna_ratings, method="pearson").sort_values(ascending=False).head(num_hotel+1))
    
    similar_to_hotel.reset_index(inplace=True)

    hotel_names = []
    for i in similar_to_hotel.title:
        hotel_names.append(i)
    
    hotel_names.pop(0)
    
    return hotel_names

In [80]:
hotel_recommender('Ankara Prenses Hotel', 4)

['Volley Hotel Ankara', 'Yayla Club', 'Aden Apart', 'Hotel2017']

In [81]:
hotel_recommender('Ankara Prenses Hotel', 10)

['Volley Hotel Ankara',
 'Yayla Club',
 'Aden Apart',
 'Hotel2017',
 'Ankara Trend House Apart',
 'Hotel 2000 Kavaklıdere',
 'Asrin Park Hotel',
 'Alfin Hotel',
 'Double Comfort Hotel',
 'Warwick Ankara']

In [82]:
hotel_recommender('Eyna Hotel', 10)

['Otto Loft Premium Apartments',
 'Izzotel Ankara',
 'Apart Hotel Best',
 'Rescate Boutique Hotel',
 'Real Hotel',
 'LUGAL',
 'No 19 Boutique Hotel',
 'Ankara Alegria Business Hotel',
 'Ankara Gold Otel',
 'Ankacity Suit Flat']